In [1]:
%load_ext autoreload
%autoreload 2
import generator
import featuretools as ft
import pandas as pd
import os
from glob import glob
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

### Несколько таблиц и есть отношения между нимим

In [2]:
parameters = {
    'path_data': '/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data',
    'tables': {
        'app.csv': 'SK_ID_CURR',
        'bureau_balance.csv': False,
        'bureau.csv': 'SK_ID_BUREAU',
        'previous.csv': 'SK_ID_PREV',
        'cash.csv': False,
        'installments.csv': False,
        'credit.csv': False
    },  # в качестве значений указываются уникальные индексы для таблиц
    'main_table': 'app.csv',  # у main_table ОБЯЗАТЕЛЬНО должен быть уникальный идентификатор
    'target': 'TARGET',  # таргет в main_table
    'depth': 2,
    'relations': [
        [('app.csv', 'SK_ID_CURR'), ('bureau.csv', 'SK_ID_CURR')],
        [('bureau.csv', 'SK_ID_BUREAU'), ('bureau_balance.csv', 'SK_ID_BUREAU')], 
        [('app.csv','SK_ID_CURR'), ('previous.csv','SK_ID_CURR')], 
        [('previous.csv', 'SK_ID_PREV'), ('cash.csv', 'SK_ID_PREV')], 
        [('previous.csv', 'SK_ID_PREV'), ('installments.csv', 'SK_ID_PREV')], 
        [('previous.csv', 'SK_ID_PREV'), ('credit.csv', 'SK_ID_PREV')],
    ],
    'time_indecies': {
        'app.csv': False,
        'bureau_balance.csv': 'bureau_balance_date',
        'bureau.csv': False,
        'cash.csv': False,
        'credit.csv': 'credit_balance_date',
        'installments.csv': 'installments_paid_date', 
        'previous.csv': False,
    },
    'time_variables': {
        'app.csv': False,
        'bureau_balance.csv': ['bureau_balance_date'],
        'bureau.csv': ['bureau_credit_application_date', 'bureau_credit_end_date', 'bureau_credit_close_date', 'bureau_credit_update_date'],
        'cash.csv': ['cash_balance_date'],
        'credit.csv': ['credit_balance_date'],
        'installments.csv': ['installments_due_date', 'installments_paid_date'], 
        'previous.csv': ['previous_decision_date', 'previous_drawing_date', 'previous_first_due_date', 'previous_last_duefirst_date'],
    },  # после получения от Ани таблицы с типами, удалить этот словарь, в этих списках должны быть поля time_index'
    'sep': ',',  # разделитель в csv файлах
    'chunk_size': 0.5,  # количество строчек обрабатываемых за момент времени (если число от 0 до 1 - то это процент строчек), рекомендуется использовать 1/n_jobs
    'max_features': 10**5,  # максимальное количество генерируемых фич
    'n_jobs': 2,  # количество потоков, в которых запускается генерация фичей
    'agg_primitives': ['count', 'max', 'min', 'mean', 'num_unique', 'percent_true', 'all', 'time_since_last'],  # добавлены примитивы для работы с датами
#     'agg_primitives': ['max'], # 'min'],
#     'trans_primitives': ['percentile'],
    'trans_primitives': ['percentile', 'not', 'cum_sum', 'cum_max', 'cum_mean', 'cum_min', 'month', 'day', 'weekday', 'week', 'time_since_previous'],  # добавлены примитивы для работы с датами
    'output_file_name': 'feature_matrix_with_dates.csv'
}

In [3]:
fg = generator.FeatureGenerator(parameters)
fg

Generator(data='/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data', output_file='feature_matrix_with_dates.csv', object_id=140611892833696)

In [4]:
%%time
fg.create_dataframes()

CPU times: user 1min 21s, sys: 6.86 s, total: 1min 28s
Wall time: 1min 29s


In [5]:
fg.check_cycles()

In [6]:
%%time
fg.create_entityset()

CPU times: user 49.5 s, sys: 7.75 s, total: 57.2 s
Wall time: 59.4 s


In [7]:
%%time
fg.create_relations()

CPU times: user 2.74 ms, sys: 142 µs, total: 2.88 ms
Wall time: 2.81 ms


In [8]:
fg.entities

Entityset: entities
  Entities:
    app.csv [Rows: 153755, Columns: 122]
    bureau_balance.csv [Rows: 7361132, Columns: 4]
    bureau.csv [Rows: 732333, Columns: 17]
    previous.csv [Rows: 703690, Columns: 37]
    cash.csv [Rows: 4111540, Columns: 9]
    installments.csv [Rows: 5267838, Columns: 9]
    credit.csv [Rows: 1180017, Columns: 24]
  Relationships:
    bureau.csv.SK_ID_CURR -> app.csv.SK_ID_CURR
    bureau_balance.csv.SK_ID_BUREAU -> bureau.csv.SK_ID_BUREAU
    previous.csv.SK_ID_CURR -> app.csv.SK_ID_CURR
    cash.csv.SK_ID_PREV -> previous.csv.SK_ID_PREV
    installments.csv.SK_ID_PREV -> previous.csv.SK_ID_PREV
    credit.csv.SK_ID_PREV -> previous.csv.SK_ID_PREV

In [9]:
# %%time
# fg.plot_entityset()

In [9]:
%%time
fg.feature_matrix_base_parallel()

Built 2462 features
Elapsed: 00:00 | Progress:   0%|          

distributed.scheduler - WARNING - Communication failed during replication: {'status': 'missing-data', 'keys': {'bytes-9f62456b1f0776c99c910c0ca859423c': ('tcp://127.0.0.1:49241',)}}


EntitySet scattered to 2 workers in 218 seconds
Elapsed: 30:28 | Progress:  95%|█████████▌

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


Elapsed: 31:01 | Progress: 100%|██████████
CPU times: user 9min 59s, sys: 2min 2s, total: 12min 1s
Wall time: 43min 13s


In [12]:
for i in fg.feature_names:
    if 'month' in str(i).lower():
        print(i)

<Feature: NUM_UNIQUE(bureau.csv.MONTH(bureau_credit_application_date))>
<Feature: NUM_UNIQUE(bureau.csv.MONTH(bureau_credit_close_date))>
<Feature: NUM_UNIQUE(bureau.csv.MONTH(bureau_credit_end_date))>
<Feature: NUM_UNIQUE(bureau.csv.MONTH(bureau_credit_update_date))>
<Feature: NUM_UNIQUE(previous.csv.MONTH(previous_decision_date))>
<Feature: NUM_UNIQUE(previous.csv.MONTH(previous_drawing_date))>
<Feature: NUM_UNIQUE(previous.csv.MONTH(previous_first_due_date))>
<Feature: NUM_UNIQUE(previous.csv.MONTH(previous_last_due_date))>
<Feature: NUM_UNIQUE(previous.csv.MONTH(previous_last_duefirst_date))>
<Feature: NUM_UNIQUE(previous.csv.MONTH(previous_termination_date))>


In [11]:
# pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/feature_matrix.csv', sep=',')[:100]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PERCENTILE(MAX(previous.csv.AMT_DOWN_PAYMENT)),PERCENTILE(MAX(previous.csv.AMT_GOODS_PRICE)),PERCENTILE(MAX(previous.csv.CNT_PAYMENT)),PERCENTILE(MAX(previous.csv.HOUR_APPR_PROCESS_START)),PERCENTILE(MAX(previous.csv.NFLAG_INSURED_ON_APPROVAL)),PERCENTILE(MAX(previous.csv.NFLAG_LAST_APPL_IN_DAY)),PERCENTILE(MAX(previous.csv.RATE_DOWN_PAYMENT)),PERCENTILE(MAX(previous.csv.RATE_INTEREST_PRIMARY)),PERCENTILE(MAX(previous.csv.RATE_INTEREST_PRIVILEGED)),PERCENTILE(MAX(previous.csv.SELLERPLACE_AREA))
0,100002,1.0,Cash loans,M,N,Y,0.0,202500.0,406597.5,24700.5,...,0.140802,0.455565,0.618042,0.047306,0.23586,0.500058,0.140802,NaN,NaN,0.644418
1,100003,0.0,Cash loans,F,N,N,0.0,270000.0,1293502.5,35698.5,...,0.547562,0.880603,0.313610,0.733489,0.73586,0.500058,0.380773,NaN,NaN,0.779176
2,100004,0.0,Revolving loans,M,Y,Y,0.0,67500.0,135000.0,6750.0,...,0.462461,0.014877,0.011732,0.003386,0.23586,0.500058,0.802941,NaN,NaN,0.163269
3,100006,0.0,Cash loans,F,N,Y,0.0,135000.0,312682.5,29686.5,...,0.979800,0.834515,0.864853,0.491319,0.23586,0.500058,0.821436,NaN,NaN,0.993468
4,100007,0.0,Cash loans,M,N,Y,0.0,121500.0,513000.0,21865.5,...,0.390932,0.579684,0.864853,0.491319,0.73586,0.500058,0.852939,NaN,NaN,0.756059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100113,0.0,Cash loans,M,Y,N,0.0,76500.0,135000.0,6696.0,...,0.346355,0.220854,0.313610,0.381555,0.73586,0.500058,0.637118,NaN,NaN,0.312153
96,100114,0.0,Cash loans,F,N,Y,0.0,135000.0,284400.0,22599.0,...,0.140802,0.185301,0.313610,0.491319,0.73586,0.500058,0.140802,NaN,NaN,0.532674
97,100115,0.0,Cash loans,F,N,N,0.0,90000.0,315000.0,14683.5,...,0.140802,0.367095,0.052537,0.284914,0.23586,0.500058,0.140802,NaN,NaN,0.265112
98,100116,0.0,Cash loans,F,N,Y,0.0,243000.0,900000.0,32017.5,...,0.488569,0.849938,0.864853,0.491319,0.73586,0.500058,0.644588,NaN,NaN,0.923650


In [12]:
# fg.feature_names

In [10]:
# def date_parser(date_in_str):
#     return dt.datetime.strptime(date_in_str, '%Y-%m-%d').date()
# date_parser('2016-02-14')
# df = pd.read_csv('data/bureau_balance.csv', sep=',', encoding="utf-8", low_memory=False, parse_dates=['bureau_balance_date'], date_parser=date_parser)
# df.head()

### Одна таблица

In [13]:
# %%time
# app = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/app.csv', sep=',')
# bureau_balance = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/bureau_balance.csv', sep=',')
# bureau = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/bureau.csv', sep=',')
# cash = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/cash.csv', sep=',')
# credit = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/credit.csv', sep=',')
# installments = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/installments.csv', sep=',')
# previous = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/previous.csv', sep=',')

In [14]:
# previous[['previous_decision_date', 'previous_drawing_date', 'previous_first_due_date', 'previous_last_duefirst_date']].head()

In [35]:
app.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1.0,Cash loans,M,N,Y,0.0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0.0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0.0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0.0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0.0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
parameters = {
    'path_data': '/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data',
    'tables': {
        'bureau_balance.csv': False,
    }, # в качестве значений указываются уникальные индексы для таблиц
    'main_table': 'bureau_balance.csv', # у main_table ОБЯЗАТЕЛЬНО должен быть уникальный идентификатор
    'target': 'STATUS',
    'depth': 2,
    'relations': [],
    'time_indecies': {
        'bureau_balance.csv': 'bureau_balance_date'
    },
    'sep': ',', # разделитель в csv файлах
    'chunk_size': 0.5, # количество строчек обрабатываемых за момент времени (если число от 0 до 1 - то это процент строчек)
    'max_features': 10**5, # максимальное количество генерируемых фич
    'n_jobs': 2, # количество потоков, в которых запускается генерация фичей
#     'agg_primitives': ['count', 'max', 'min', 'median', 'mean', 'num_unique', 'sum'],
    'agg_primitives': ['max'], # 'min'],
    'trans_primitives': ['year', 'day', 'weekday', 'month'], # 'cum_sum'],
    'output_file_name': 'test_bb_date.csv'
}

In [19]:
fg = generator.FeatureGenerator(parameters)
fg.create_dataframes()
fg.create_entityset()

In [20]:
fg.entities

Entityset: entities
  Entities:
    bureau_balance.csv [Rows: 7361132, Columns: 4]
  Relationships:
    No relationships

In [10]:
fg.plot_entityset()

In [21]:
fg.drop_contains

['STATUS)', 'bureau_balance_index']

In [22]:
%%time
fg.feature_matrix_base_parallel()

Built 6 features
EntitySet scattered to 2 workers in 12 seconds
Elapsed: 00:42 | Progress: 100%|██████████
CPU times: user 30.5 s, sys: 5.94 s, total: 36.4 s
Wall time: 1min 16s


In [23]:
fg.feature_names

[<Feature: SK_ID_BUREAU>,
 <Feature: STATUS>,
 <Feature: DAY(bureau_balance_date)>,
 <Feature: MONTH(bureau_balance_date)>,
 <Feature: WEEKDAY(bureau_balance_date)>,
 <Feature: YEAR(bureau_balance_date)>]

In [24]:
df = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/test_bb_date.csv', sep=',')

In [25]:
df.head()

,bureau_balance_index,SK_ID_BUREAU,STATUS,DAY(bureau_balance_date),MONTH(bureau_balance_date),WEEKDAY(bureau_balance_date),YEAR(bureau_balance_date)
0,811015,6318773,X,28,2,2,2018
1,853645,6551168,X,28,2,2,2018
2,1079910,5788299,0,28,2,2,2018
3,1109416,5901079,0,28,2,2,2018
4,1395670,5092063,X,28,2,2,2018


In [26]:
bb = pd.read_csv('/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/bureau_balance.csv', sep=',')

In [31]:
bb[bb['SK_ID_BUREAU'] == 5092063].sort_values(['bureau_balance_date']).head()

,SK_ID_BUREAU,STATUS,bureau_balance_date
1395670,5092063,X,2018-02-28
1395669,5092063,X,2018-03-07
1395668,5092063,0,2018-03-14
1395667,5092063,X,2018-03-21
1395666,5092063,0,2018-03-28


In [4]:
pd.set_option('max_colwidth', 1000)

In [5]:
primitives = ft.list_primitives()
print(primitives.shape)

primitives[primitives['type'] == 'aggregation']

(79, 5)


,name,type,dask_compatible,koalas_compatible,description
0,first,aggregation,False,False,Determines the first value in a list.
1,n_most_common,aggregation,False,False,Determines the `n` most common elements.
2,count,aggregation,True,True,"Determines the total number of values, excluding `NaN`."
3,num_true,aggregation,True,False,Counts the number of `True` values.
4,num_unique,aggregation,True,True,"Determines the number of distinct values, ignoring `NaN` values."
5,skew,aggregation,False,False,Computes the extent to which a distribution differs from a normal distribution.
6,min,aggregation,True,True,"Calculates the smallest value, ignoring `NaN` values."
7,mean,aggregation,True,True,Computes the average for a list of values.
8,time_since_first,aggregation,False,False,Calculates the time elapsed since the first datetime (in seconds).
9,all,aggregation,True,False,Calculates if all values are 'True' in a list.


In [7]:
primitives[primitives['type'] == 'transform']

,name,type,dask_compatible,koalas_compatible,description
22,less_than_equal_to,transform,True,True,Determines if values in one list are less than or equal to another list.
23,absolute,transform,True,True,Computes the absolute value of a number.
24,scalar_subtract_numeric_feature,transform,True,True,Subtract each value in the list from a given scalar.
25,divide_by_feature,transform,True,True,Divide a scalar by each value in the list.
26,greater_than_scalar,transform,True,True,Determines if values are greater than a given scalar.
27,is_null,transform,True,True,Determines if a value is null.
28,latitude,transform,False,False,Returns the first tuple value in a list of LatLong tuples.
29,not,transform,True,True,Negates a boolean value.
30,add_numeric_scalar,transform,True,True,Add a scalar to each value in the list.
31,week,transform,True,True,Determines the week of the year from a datetime.
